装饰器

# 例子引入
## 汉堡

In [1]:
def meat(food = '--鸡肉饼--'):
    print(food)
    
burger = meat
burger()

--鸡肉饼--


In [5]:
def vegetable(func):
    def wrapper():
        print(' #西红柿#')
        func()
        print(' ~沙拉菜~')
    return wrapper

burger = vegetable(meat)
burger()

 #西红柿#
--鸡肉饼--
 ~沙拉菜~


In [6]:
def bread(func):
    def wrapper():
        print('</------\>')
        func()
        print('<\------/>')
    return wrapper

burger = bread(vegetable(meat))
burger()

</------\>
 #西红柿#
--鸡肉饼--
 ~沙拉菜~
<\------/>


**要点**：面包和蔬菜「装饰」着鸡肉饼，bread() 和 vegatable() 这两个函数起着「装饰器」的作用，它们没有改变 meat() 函数，只在它的基础上添砖加瓦，最后把鸡肉饼装饰成汉堡。

## 装饰器用法

下面是装饰器的正规语法，用 @func 语法 (注意@符)，将@bread 和 @vegatable 放在要装饰的函数上面。

In [8]:
@bread
@vegetable
def meat(food='--鸡肉饼--'):
    print(food)
    
burger = meat
burger()

</------\>
 #西红柿#
--鸡肉饼--
 ~沙拉菜~
<\------/>


装饰器是有序的，如下例所示，如果互换 bread() 和 vegatable() 这两函数的位置，那么这汉堡最外层是蔬菜，中间是面包，里面是鸡肉饼，不像汉堡了。

In [9]:
@vegetable
@bread
def meat(food='--鸡肉饼--'):
    print(food)
    
burger = meat
burger()

 #西红柿#
</------\>
--鸡肉饼--
<\------/>
 ~沙拉菜~


**要点**：一个函数可以被多个装饰器装饰，装饰器的顺序很重要。

# 函数

在 Python 里函数是「一等公民」，我们可以
1. 把函数赋值给变量
2. 在函数里定义函数
3. 在函数里返回函数
4. 把函数传递给函数

# 装饰器

## 闭包到装饰器

In [1]:
def outer_func(msg):
    def inner_func():
        print(msg)
    return inner_func

内部函数 inner_func() 可以使用外部函数 outer_func() 的参数 msg (注意 msg 不在自己定义范围内)，最后返回内部函数的对象 inner_func。

In [12]:
hi_func = outer_func('Hi')
bye_func = outer_func('Bye')

接下来，我们
1. 将 outer_func() 改成 decorator_func()
2. 将 inner_func() 改成 wrapper_func()

In [13]:
def decorator_func(msg):
    def wrapper_func():
        print(msg)
    return wrapper_func

这就是装饰器，它返回 wrapper_func 对象，随时等着被调用，一旦被调用就运行 print(msg) 而打印出 msg。

等等，严格来说，对于装饰器，参数是函数而不是变量 (1.4 节讲了函数可以当成参数传递给另一个函数)。

In [2]:
def decorator_func(func):
    def wrapper_func():
        return func()
    return wrapper_func

## 装饰器初体验

In [3]:
def display():
    print('Run display function')    

decorated_display = decorator_func(display)
decorated_display()

Run display function


但装饰器的特性是给原函数做装饰，但不改变原函数里的内容，比如下面代码第 3 行，我们希望在运行原函数 func() 之前，输出原函数的名字 (用 __name__属性)。

In [17]:
def decorator_func(func):
    def wrapper_func():
        print('Executed before {}'.format(func.__name__))
        return func()
    return wrapper_func

decorated_display = decorator_func(display)
decorated_display()

Executed before display
Run display function


但是，每次这样调用装饰器太过繁琐。Python 里有一种等价语法。把 @decorator_func 写在被装饰的函数上面即可，代码如下。

In [18]:
@decorator_func
def display():
    print('Run display function')

等价于：（语法糖）

In [ ]:
display = decorator_func(display)

In [19]:
display()

Executed before display
Run display function


## 装饰器知识点

在本节我们了解几个装饰器的知识点：
1. 多个装饰器来装饰一个函数
2. 传递参数给装饰函数 (wrapper function)
3. functools.wraps 的用法
4. 传递参数装饰器 (decorator) 

### 多个装饰器

In [20]:
def slogan():
    return 'I love Python'

In [21]:
def uppercase_decorator(func):
    def wrapper():
        return func().upper()
    return wrapper

In [22]:
def split_decorator(func):
    def wrapper():
        return func().split()
    return wrapper

In [23]:
@split_decorator
@uppercase_decorator
def slogan():
    return 'I love Python'

In [24]:
slogan()

['I', 'LOVE', 'PYTHON']

等价于

In [25]:
slogan = split_decorator(uppercase_decorator(slogan))

### 传递参数给装饰函数 (wrapper function)

首先看一个没有参数的 wrapper() 的例子。 

In [26]:
def my_logger(func):
    def wrapper():
        name = func.__name__
        print('Before calling {}'.format(name))
        func()
        print('After calling {}'.format(name))
    return wrapper

@my_logger
def func():
    print('calling func')
    
func()

Before calling func
calling func
After calling func


In [27]:
def my_logger(func):
    def wrapper( arg1, arg2 ):
        name = func.__name__
        print('Before calling {}'.format(name))
        result = func( arg1, arg2 )
        print('After calling {}'.format(name))
        return result
    return wrapper

@my_logger
def add(a,b):
    return a+b

add(1,2)

Before calling add
After calling add


3

In [28]:
def my_logger(func):
    def wrapper( *args ):
        name = func.__name__
        print('Before calling {}'.format(name))
        result = func( *args )
        print('After calling {}'.format(name))
        return result
    return wrapper

In [29]:
@my_logger
def add(a,b,c):
    return a+b+c

add(1,2,3)

Before calling add
After calling add


6

In [31]:
@my_logger
def add(a,b,c,d,e):
    return a+b+c+d+e

add(1,2,3,4,5)

Before calling add
After calling add


15

In [32]:
def my_logger(func):
    def wrapper( *args, **kwargs ):
        name = func.__name__
        print('Before calling {}'.format(name))
        result = func( *args, **kwargs )
        print('After calling {}'.format(name))
        return result
    return wrapper

### functools.wraps 的用法

在装饰器里，装饰之后的函数的名称会弄乱。比如一个简单的函数 f()，它的名称就是 f (用 __name__)。

In [33]:
def f():
    pass

print(f.__name__)

f


In [34]:
def decorator(func):
    def wrapper():
        func()
    return wrapper

@decorator
def f():
    pass

print(f.__name__)

wrapper


In [35]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper():
        return func()
    return wrapper

@decorator
def f():
    pass

print(f.__name__)

f


### 传递参数装饰器 (decorator) 

In [36]:
def money_format(func):
    def wrapper(*args, **kwargs):
        r = func(*args, **kwargs)
        formatted = '{:.2f}'.format(r)
        return formatted
    return wrapper

In [38]:
@money_format
def add( PV1, PV2 ):
    return PV1 + PV2

add(100, 100.4545)

'200.45'

In [39]:
def currency_unit(curr):
    def money_format(func):
        def wrapper(*args, **kwargs):
            r = func(*args, **kwargs)
            formatted = '{:.2f}'.format(r) + ' ' + curr
            return formatted
        return wrapper
    return money_format


@currency_unit('USD')
def add( PV1, PV2 ):
    return PV1 + PV2

add(100, 100.4545)

'200.45 USD'

In [41]:
@currency_unit('CNY')
def add( PV1, PV2, PV3 ):
    return PV1 + PV2 + PV3

add(100, 100, 100.4545)

'300.45 CNY'

# 总结

装饰器就是「接受函数为参数」并「返回函数为输出」的函数。

1. 装饰器不会更改参数函数里的内容。
2. 装饰器返回的其实是函数对象。
3. 装饰器本质就是个函数。

就装饰器定义里面有这么多函数出现，要想理解装饰器，一定要理解好函数 (本贴第 1 节的内容)。